# Machine Learning Assessment

## Input

Import calls.

In [41]:
import pandas as pd

# Convert to pandas DataFrame
calls_df = pd.read_csv('./data/calls.csv')
print(calls_df.info())
print(calls_df.head(1))

calls_df['Datetime'] = pd.to_datetime(calls_df['Datetime'],\
                                      format="%m/%d/%Y %I:%M:%S %p").dt.tz_localize(tz='US/Pacific', ambiguous='NaT')
calls_df.dropna(inplace=True)
calls_df.set_index('Datetime', inplace=True)
calls_df.sort_index(inplace=True)

# results_df['type'] = results_df['type'].astype('category')

print(calls_df.info())
print(calls_df.head(2))
print(calls_df.tail(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1561260 entries, 0 to 1561259
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Address          1561199 non-null  object 
 1   Type             1561260 non-null  object 
 2   Datetime         1561260 non-null  object 
 3   Latitude         1560622 non-null  float64
 4   Longitude        1560622 non-null  float64
 5   Report Location  1560652 non-null  object 
 6   Incident Number  1561260 non-null  object 
dtypes: float64(2), object(5)
memory usage: 83.4+ MB
None
      Address      Type                Datetime  Latitude   Longitude  \
0  904 7th Av  Car Fire  05/22/2019 12:55:00 PM   47.6069 -122.329051   

               Report Location Incident Number  
0  POINT (-122.329051 47.6069)      F190051945  
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1560227 entries, 2003-11-07 09:30:48-08:00 to 2021-05-17 08:25:00-07:00
Data columns (total 6 column

In [23]:
wtr_df = pd.read_csv('data/Seattle Weatherdata 2002 to 2020.csv')
wtr_df['datetime'] = pd.to_datetime(wtr_df['dt'], unit='s').\
dt.tz_localize(tz='UTC').\
dt.tz_convert('US/Pacific')
wtr_df.set_index('datetime', inplace=True)

# weather_df['temp'].groupby(weather_df['datetime'].dt.hour).mean().plot()\n",
# plt.show()\n",

# weather_df['pressure'].groupby(weather_df['datetime'].dt.hour).mean().plot()\n",
# plt.show()\n",

print(wtr_df.info())
# print(weather_df.head())\n",
# print(weather_df.tail())\n",
# wtr_df[wtr_df.index.duplicated()]\n",
# wtr_df.loc[\"2002-01-08 11:00:00-08:00\"]"


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 170996 entries, 2001-12-31 16:00:00-08:00 to 2020-11-04 15:00:00-08:00
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   dt                   170996 non-null  int64  
 1   dt_iso               170996 non-null  object 
 2   timezone             170996 non-null  int64  
 3   city_name            170996 non-null  object 
 4   lat                  170996 non-null  float64
 5   lon                  170996 non-null  float64
 6   temp                 170996 non-null  float64
 7   feels_like           170996 non-null  float64
 8   temp_min             170996 non-null  float64
 9   temp_max             170996 non-null  float64
 10  pressure             170996 non-null  int64  
 11  sea_level            0 non-null       float64
 12  grnd_level           0 non-null       float64
 13  humidity             170996 non-null  int64  
 14  wind_speed           1

In [44]:
tar = calls_df['incident_number'].resample('H').count().to_frame('incident_count')
tar

,incident_count
datetime,
2003-11-07 09:00:00-08:00,5
2003-11-07 10:00:00-08:00,20
2003-11-07 11:00:00-08:00,5
2003-11-07 12:00:00-08:00,16
2003-11-07 13:00:00-08:00,13
...,...
2020-11-11 05:00:00-08:00,0
2020-11-11 06:00:00-08:00,0
2020-11-11 07:00:00-08:00,0


In [55]:
#!/usr/bin/env python

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt



ftr = wtr_df[['temp', 'pressure', 'humidity']].copy()
ftr.drop_duplicates(inplace=True)
ftr = ftr.join(ftr.index.isocalendar().join(pd.DataFrame(ftr.index.hour)))
ftr

,temp,pressure,humidity,year,week,day,datetime
datetime,,,,,,,
2001-12-31 16:00:00-08:00,9.21,1019,71,2002,1,1,NaN
2001-12-31 17:00:00-08:00,8.20,1019,81,2002,1,1,NaN
2001-12-31 18:00:00-08:00,6.61,1019,86,2002,1,1,NaN
2001-12-31 19:00:00-08:00,6.43,1018,87,2002,1,1,NaN
2001-12-31 20:00:00-08:00,6.25,1018,81,2002,1,1,NaN
...,...,...,...,...,...,...,...
2020-11-04 10:00:00-08:00,15.48,1016,89,2020,45,3,NaN
2020-11-04 11:00:00-08:00,16.36,1016,77,2020,45,3,NaN
2020-11-04 13:00:00-08:00,17.28,1016,72,2020,45,3,NaN


In [56]:
pd.DataFrame(ftr.index.hour).set_index(ftr.index)

AttributeError: 'Index' object has no attribute 'hour'